In [5]:
from datasets import Dataset
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
import pandas as pd

DATA PREP

In [6]:
train_df = pd.read_csv('data/train_frame.csv')
test_df = pd.read_csv('data/test_frame.csv')

In [7]:
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

In [8]:
train_df.head()

,text,label,label_text
0,* slick-looking design and improved interface,1,positive
1,the day finally arrived when i was sure i 'd l...,0,negative
2,"as for bluetooth , no problems at all .",1,positive
3,2 ) storage capacity,1,positive
4,neither message was answered ( they ask for 24...,0,negative


MODEL

In [12]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=32,
    num_epochs=3,
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [13]:
# Train and evaluate!
trainer.train()
metrics = trainer.evaluate()

***** Running training *****
  Num examples = 1600
  Num epochs = 3
  Total optimization steps = 50
  Total train batch size = 32
Epoch: 100%|██████████| 3/3 [01:21<00:00, 27.32s/it]
***** Running evaluation *****


In [14]:
metrics = trainer.evaluate()
metrics

***** Running evaluation *****


{'accuracy': 0.9110225763612217}

In [17]:
from sklearn.metrics import classification_report
pred1 = trainer.model.predict(test_df.text)
print(classification_report(test_df.label, pred1, target_names=['negative', 'positive']))

              precision    recall  f1-score   support

    negative       0.91      0.85      0.88       286
    positive       0.91      0.95      0.93       467

    accuracy                           0.91       753
   macro avg       0.91      0.90      0.90       753
weighted avg       0.91      0.91      0.91       753



In [17]:
trainer.model.predict(['it is awesome', 'it is very bad'])

array([1, 0])

In [20]:
trainer.model.save_pretrained('sample_sentiment_model')